In [1]:
from typing import List

import numpy as np
from numpy.typing import NDArray
from scipy.optimize import Bounds, minimize

In [2]:
from model.tableaux import Tableaux
from model.priors import Gaussian, Priors

In [3]:
class GaussianRegularizer:
    mu: NDArray[np.float64]  # (p,)
    sig_sq: NDArray[np.float64]  # (p,)

    def __init__(self, priors: Priors, keys: List[str]):
        gaussians = [priors.get(k) for k in keys]
        self.mu = np.array([g.mean for g in gaussians])
        self.sig_sq = np.array([g.var for g in gaussians])

    def score(self, beta):
        """returns -logP"""
        return (0.5 * np.square(beta - self.mu) / self.sig_sq).sum()

    def grad(self, beta):
        """returns d/db (-logP)"""
        return (beta - self.mu) / self.sig_sq

In [4]:
def objective(beta, tbx: Tableaux, reg: GaussianRegularizer):
    return tbx.score(beta) + reg.score(beta)

def gradient(beta, tbx: Tableaux, reg: GaussianRegularizer):
    return tbx.grad(beta) + reg.grad(beta)

In [6]:
DATA_FILE = "data/JapaneseLoanword.txt"
tbx = Tableaux.load(DATA_FILE)

CONSTRAINT_FILE = "data/JLConstraint.txt"
cons = Priors(filename=CONSTRAINT_FILE)
# cons = Priors(default=Gaussian(-2, 10))
reg = GaussianRegularizer(cons, tbx.feature_names)

beta0 = np.array([-5.0] * len(tbx.features))

In [8]:
# 20111.58649332986
res = minimize(objective, beta0, args=(tbx, reg), jac=gradient,
         bounds=Bounds(-np.inf, 0.), tol=1e-10)
res

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 693.1473318076597
        x: [-3.460e+01 -1.730e+01 -1.730e+01]
      nit: 28
      jac: [-1.321e-04  1.576e-04  1.576e-04]
     nfev: 31
     njev: 31
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>

In [14]:
ex = tbx.examples[2]
ex.predict(res.x), ex.freqs / sum(ex.freqs)

(array([5.00000190e-01, 4.99999810e-01, 4.72557362e-16]),
 array([0.5, 0.5, 0. ]))

In [13]:
objective(np.array([
    -0.03173206593057489, -0.15883115214362092
]), tbx, reg)

3.4129236084622177

In [378]:
def softmax(x):
    a = np.exp(x)
    return a / a.sum()

In [379]:
-res.x

array([5.4937805 , 0.69021848, 1.02176443, 5.00423239])

In [380]:
res.x

for ex in tbx.examples:
    print(softmax(ex.violations @ res.x), ex.freqs / ex.freqs.sum())

[0.54999992 0.45000008] [0.55 0.45]
[0.62999989 0.37000011] [0.63 0.37]
[0.38000003 0.61999997] [0.38 0.62]
